# this is where I got it from

https://bpostance.github.io/posts/clustering-mixed-data/

In [29]:
from __future__ import print_function
import time
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn import preprocessing
from numpy import asarray
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import scale,RobustScaler,StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans
from matplotlib import cm
from sklearn.manifold import TSNE
from sklearn.datasets import make_blobs
from sklearn.metrics import silhouette_samples, silhouette_score

In [3]:
raw = pd.read_csv('oscars.csv')
raw.head()

,title,runtime,imdb_score,imdb_votes,user_review_count,critic_review_count,critic_overall_score,critic_positive_score,critic_mixed_score,critic_negative_score,user_overall_score,user_positive_score,user_mixed_score,user_negative_score,year_film,year_ceremony,ceremony,category,name,winner
0,The Jazz Singer,88,6.4,9888,113.0,66,66,5,2,0,7.0,3,1,0,1927,1928,1,WRITING (Adaptation),Alfred Cohn,False
1,The Jazz Singer,88,6.4,9888,113.0,66,66,5,2,0,7.0,3,1,0,1952,1953,25,MUSIC (Scoring of a Musical Picture),"Ray Heindorf, Max Steiner",False
2,The Patriot,165,7.2,269439,1200.0,172,63,23,11,1,6.7,113,35,23,1928,1929,2,ACTOR,Lewis Stone,False
3,The Patriot,165,7.2,269439,1200.0,172,63,23,11,1,6.7,113,35,23,1928,1929,2,ART DIRECTION,Hans Dreier,False
4,The Patriot,165,7.2,269439,1200.0,172,63,23,11,1,6.7,113,35,23,1928,1929,2,DIRECTING,Ernst Lubitsch,False


In [5]:
# changing the 'title' as ids
le = preprocessing.LabelEncoder()
titles = list(raw['title'])
le.fit(titles)
encoded_titles = le.transform(titles)
raw['title'] = encoded_titles

In [6]:
# example of a normalization

columns = ['title','runtime', 'imdb_score', 'imdb_votes', 'user_review_count', 'critic_review_count','critic_overall_score','critic_positive_score','critic_mixed_score','critic_negative_score','user_overall_score','user_positive_score','user_mixed_score','user_negative_score','year_film','year_ceremony','ceremony']

In [7]:
# convert numerical-object to numericals
for col in columns:
    raw[col] = raw[col].replace('[^.0-9]', '', regex=True,).astype(float).fillna(0.0)


In [8]:
# clean textual classes
for col in raw.select_dtypes(include='object').columns:
        raw[col] = raw[col].str.upper().replace('Z_','',regex=True).replace('[^A-Z]','',regex=True)

In [9]:
data_types = {f:t for f,t in zip(raw.columns,raw.dtypes)}

In [13]:
# copy df and drop both 'winner' and 'title' since it will not contribute to clustering
df = raw.copy()
df = df.drop(labels=['winner'],axis=1)
df = df.drop(labels=['title'],axis=1)

In [20]:
# counts unique categories of each attribute
data_count = pd.DataFrame(df.nunique(),columns=['num'],index=None).sort_values('num').reset_index()
data_count.columns = ['name','num']
data_count['type'] = 'numerical'
print(data_count)

                     name   num       type
0   critic_negative_score    19  numerical
1      critic_mixed_score    24  numerical
2              imdb_score    45  numerical
3   critic_positive_score    45  numerical
4      user_overall_score    56  numerical
5    critic_overall_score    77  numerical
6                category    86  numerical
7                ceremony    91  numerical
8           year_ceremony    91  numerical
9               year_film    91  numerical
10                runtime   129  numerical
11       user_mixed_score   131  numerical
12    user_negative_score   134  numerical
13    critic_review_count   345  numerical
14    user_positive_score   410  numerical
15      user_review_count   583  numerical
16             imdb_votes  1343  numerical
17                   name  3008  numerical


In [21]:
#sorting the attributes as either nominal or numerical

data_count.loc[data_count['name'].isin(['category','name']),'type']='nominal'

nominal_features = list(data_count.loc[data_count['type']=='nominal','name'])
numerical_features = list(data_count.loc[data_count['type']=='numerical','name'])
print(numerical_features)

['critic_negative_score', 'critic_mixed_score', 'imdb_score', 'critic_positive_score', 'user_overall_score', 'critic_overall_score', 'ceremony', 'year_ceremony', 'year_film', 'runtime', 'user_mixed_score', 'user_negative_score', 'critic_review_count', 'user_positive_score', 'user_review_count', 'imdb_votes']


In [22]:
df[numerical_features][:2]

,critic_negative_score,critic_mixed_score,imdb_score,critic_positive_score,user_overall_score,critic_overall_score,ceremony,year_ceremony,year_film,runtime,user_mixed_score,user_negative_score,critic_review_count,user_positive_score,user_review_count,imdb_votes
0,0.0,2.0,6.4,5.0,7.0,66.0,1.0,1928.0,1927.0,88.0,1.0,0.0,66.0,3.0,113.0,9888.0
1,0.0,2.0,6.4,5.0,7.0,66.0,25.0,1953.0,1952.0,88.0,1.0,0.0,66.0,3.0,113.0,9888.0


In [23]:
# Standard Scaled "mean normalisation" for numerical_data
scaler = StandardScaler()
scaler.fit(df[numerical_features])
print("Feature Means:\t",[f"{x:.2}" for x in scaler.mean_])
numerical_data = scaler.transform(df[numerical_features])
numerical_data = pd.DataFrame(numerical_data,index=df.index,columns=numerical_features)
numerical_data[:2]

Feature Means:	 ['0.6', '3.5', '7.5', '1.6e+01', '6.6', '7.5e+01', '5.4e+01', '2e+03', '2e+03', '1.3e+02', '2.1e+01', '1.9e+01', '1.3e+02', '2.2e+02', '4.9e+02', '2e+05']


,critic_negative_score,critic_mixed_score,imdb_score,critic_positive_score,user_overall_score,critic_overall_score,ceremony,year_ceremony,year_film,runtime,user_mixed_score,user_negative_score,critic_review_count,user_positive_score,user_review_count,imdb_votes
0,-0.419553,-0.415185,-1.781635,-1.043574,0.127831,-0.693879,-2.380749,-2.423122,-2.423122,-1.463592,-0.521495,-0.40332,-0.602845,-0.42927,-0.504924,-0.561136
1,-0.419553,-0.415185,-1.781635,-1.043574,0.127831,-0.693879,-1.294738,-1.292995,-1.292995,-1.463592,-0.521495,-0.40332,-0.602845,-0.42927,-0.504924,-0.561136


In [24]:
# using MinMax scale for numerical data

scaler = MinMaxScaler(feature_range=(0,1))
scaler.fit(df[numerical_features])
numerical_data = scaler.transform(df[numerical_features])
numerical_data = pd.DataFrame(numerical_data,index=df.index,columns=numerical_features)
numerical_data[:2]

,critic_negative_score,critic_mixed_score,imdb_score,critic_positive_score,user_overall_score,critic_overall_score,ceremony,year_ceremony,year_film,runtime,user_mixed_score,user_negative_score,critic_review_count,user_positive_score,user_review_count,imdb_votes
0,0.0,0.076923,0.45098,0.111111,0.76087,0.595238,0.000000,0.000000,0.000000,0.234694,0.001825,0.0,0.09,0.000516,0.013989,0.00381
1,0.0,0.076923,0.45098,0.111111,0.76087,0.595238,0.263736,0.271739,0.271739,0.234694,0.001825,0.0,0.09,0.000516,0.013989,0.00381


In [25]:
# We will be using one-hot-coding for nominal data
nominal_features = [c for c in nominal_features]
print("Nominals:\n",nominal_features)

nominal_data = list()
for i,x in df[nominal_features].nunique().iteritems():
    if x <= 2:
        nominal_data.append(pd.get_dummies(df[[i]],drop_first=True))
    elif x > 2:
        nominal_data.append(pd.get_dummies(df[[i]],drop_first=False))
        
nominal_data = pd.concat(nominal_data,axis=1)

Nominals:
 ['category', 'name']


In [34]:
# transformed and scaled dataset
Xy_scaled = pd.concat([numerical_data,nominal_data],axis=1)
print(f'Data min:max {Xy_scaled.min().min(),Xy_scaled.max().max()}')

# original data
Xy_original = df[numerical_features+nominal_features].copy()

Data min:max (0.0, 1.0000000000000002)


In [105]:
column_names = Xy_scaled.columns.values
column_names.shape

(3110,)

In [66]:
#reducing dimension using PCA.
#calculates the % of the data we still have of our original
pca_2 = PCA(n_components=100)
pca_2result=pca_2.fit_transform(Xy_scaled)
print('Explained variation per principal component: {}'.format(pca_2.explained_variance_ratio_))
print('Cumulative variance explained by 2 principal components: {:.2%}'.format(np.sum(pca_2.explained_variance_ratio_)))

Explained variation per principal component: [0.10010547 0.04092751 0.02527659 0.02374137 0.02295355 0.0221212
 0.02125963 0.01933648 0.01783559 0.01706261 0.01641692 0.01592568
 0.01490599 0.01474932 0.0139712  0.01300878 0.01082706 0.01059976
 0.00972705 0.00900673 0.00744774 0.00727505 0.00704854 0.00663187
 0.00611777 0.00579964 0.00546556 0.00509528 0.00500401 0.00466732
 0.00459666 0.00453074 0.004476   0.00441772 0.0042229  0.00418468
 0.00404746 0.00396609 0.0038491  0.00376894 0.00372875 0.00353339
 0.00339174 0.00322813 0.00289736 0.0027201  0.0025632  0.00242232
 0.00228483 0.00215702 0.00212595 0.00208665 0.00198706 0.0017824
 0.00133069 0.00129623 0.00127633 0.00123161 0.00121046 0.00118605
 0.00113991 0.00113029 0.0011105  0.00109189 0.00107242 0.00106292
 0.00105087 0.00104327 0.000995   0.00098847 0.00094548 0.00093373
 0.00091786 0.00087208 0.00086125 0.00082933 0.0008223  0.00076813
 0.00076711 0.00075614 0.00072957 0.00072939 0.00071926 0.00071522
 0.0006969  0.00067

In [140]:
#Checking which features are important using PCA
#use components method: number of principal components as rows
#The larger the absolute value the more important it is.

#We are picking out k columns which are most important

index = []
for i in range(100):
    index.append('PC_%d'%int(i))
    

dataset_pca = pd.DataFrame(abs(pca_2.components_), columns=Xy_scaled.columns, index=index)
#print('\n\n', dataset_pca)

#print(Xy_scaled[column_names[1]].mean())

column_avg =[]
for i in range(len(column_names)):
    column_avg.append(Xy_scaled[column_names[i]].mean())

#print(column_avg)
#print(np.argmax(column_avg))
k = 5
top_k=np.argpartition(column_avg,-k)[-k:]

for k in top_k:
    print(column_avg[k],k)

0.5826400071569192 8
0.5826400071569194 7
0.7211859605176804 4
0.6636855931215498 2
0.7075440372765472 5


In [142]:
#Printing out important features based on the threshod of 0.3
print("\n*************** Most important features *************************")
for i in range(len(index)):
    print('As per PC %d:\n'%i, (dataset_pca[dataset_pca > 0.3].iloc[i]).dropna(),'\n\n')



*************** Most important features *************************
As per PC 0:
 critic_positive_score    0.352718
user_overall_score       0.356180
ceremony                 0.458028
year_ceremony            0.453493
year_film                0.453493
Name: PC_0, dtype: float64 


As per PC 1:
 user_overall_score      0.628246
critic_overall_score    0.338728
Name: PC_1, dtype: float64 


As per PC 2:
 category_DIRECTING      0.761833
category_FILMEDITING    0.559730
Name: PC_2, dtype: float64 


As per PC 3:
 user_overall_score                 0.370198
category_ACTORINASUPPORTINGROLE    0.408607
category_DIRECTING                 0.306293
category_FILMEDITING               0.704835
Name: PC_3, dtype: float64 


As per PC 4:
 critic_positive_score              0.377205
user_overall_score                 0.462622
category_ACTORINASUPPORTINGROLE    0.398744
category_DIRECTING                 0.309572
Name: PC_4, dtype: float64 


As per PC 5:
 category_ACTORINASUPPORTINGROLE      0.700512

As per PC 64:
 category_MUSICSONGORIGINALFORTHEPICTURE    0.63944
category_WRITINGORIGINALSTORY              0.56835
Name: PC_64, dtype: float64 


As per PC 65:
 category_MUSICSCORINGOFMUSICADAPTATIONORTREATMENT    0.754469
category_WRITINGORIGINALSTORY                        0.373147
Name: PC_65, dtype: float64 


As per PC 66:
 category_SPECIALVISUALEFFECTS      0.562383
name_MUSICANDLYRICBYRANDYNEWMAN    0.628359
Name: PC_66, dtype: float64 


As per PC 67:
 category_SPECIALVISUALEFFECTS      0.594609
name_MUSICANDLYRICBYRANDYNEWMAN    0.654277
Name: PC_67, dtype: float64 


As per PC 68:
 critic_negative_score    0.393353
name_COLLEENATWOOD       0.332928
name_SANDYPOWELL         0.671260
Name: PC_68, dtype: float64 


As per PC 69:
 critic_negative_score    0.536395
name_SANDYPOWELL         0.594775
Name: PC_69, dtype: float64 


As per PC 70:
 category_ASSISTANTDIRECTOR            0.632910
category_WRITINGMOTIONPICTURESTORY    0.517325
Name: PC_70, dtype: float64 


As per PC 71

In [144]:
# Hyperparameter tuning using the silhouette score method
# Silhouette score values range from 0 to 1.

# candidate values for our cluster
candidate_values = [2, 3, 4, 5, 10, 15, 20, 25, 30, 35, 40]